In [1]:
import pandas as pd
import numpy as np
import pickle,sys,os
import glob,gc
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","79g")
conf.set("spark.executor.memory", "89g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","75g")
conf.set("spark.driver.extraJavaOptions","-Xss19000m")
conf.set("spark.driver.extraJavaOptions","-Xms79g")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions",99000)
conf.set("spark.driver.maxResultSize", "20g")
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringFields", 100000)
spark.conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",99*1024*1024*1024)
import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)
spark.version

0.32.0


'3.0.0-preview2'

In [3]:
from sklearn.utils import parallel_backend
from joblibspark import register_spark
register_spark()

In [4]:
npy_save_dir = '/home/antslab/spark_data/pcap_inter/2020_01_09/中華電信/Embedding_POC/'
cluster_save_dir = '/home/antslab/spark_data/pcap_inter/2020_01_09/中華電信/Clustering_POC/'

In [5]:
all_npys = glob.glob(npy_save_dir+'/*.npy')
for i,npy in enumerate(all_npys):
    if i==0:
        all_npy = np.load(npy)
    else:
        all_npy = np.concatenate((all_npy,np.load(npy)),axis=-1)
all_npy = np.nan_to_num(all_npy)
array_sum = np.sum(all_npy)
assert np.isnan(array_sum)==False
all_npy.shape

(5538128, 124)

In [8]:
%%time
# with parallel_backend('spark'):
for i in tqdm(range(30-9)):
    cluster_num = i+9
    clusters = KMeans(init='k-means++',n_init=int(cluster_num*5/4),max_iter=350,precompute_distances='auto',algorithm='auto',n_clusters=cluster_num,n_jobs=-1,verbose=1) #,random_state=725035
    gc.collect()
    clusters.fit(all_npy)
    cluster_labels = clusters.labels_
    silhouette_avg = silhouette_score(all_npy, cluster_labels,sample_size=int(len(all_npy)/100))
    print("cluster#:",cluster_num,"score:",silhouette_avg)    
    pickle.dump(obj=clusters,file=open(cluster_save_dir+str(cluster_num)+"_"+str(silhouette_avg)+".pkl",'wb'))


  0%|          | 0/21 [16:38<?, ?it/s]


KeyboardInterrupt: 

In [20]:
print("cluster#:",cluster_num,"score:",silhouette_avg)

cluster#: 2 score: 0.16080563374774132


In [22]:
pickle.dump(obj=clusters,file=open(cluster_save_dir+"_"+str(cluster_num)+"_"+str(silhouette_avg)+".pkl",'wb'))
print("cluster#:",cluster_num,"score:",silhouette_avg)

cluster#: 3 score: 0.1344282705828418


In [12]:
%%time
cluster_labels = clusters.labels_
silhouette_avg = silhouette_score(all_npy, cluster_labels,sample_size=int(len(all_npy)/50))
print(silhouette_avg)
# plt.plot(range(2,20), silhouette_avg)

0.13657965974797404
CPU times: user 4min 21s, sys: 8min 24s, total: 12min 46s
Wall time: 2min
